In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif' 
plt.rcParams['font.serif'] = 'Ubuntu' 
plt.rcParams['font.monospace'] = 'Ubuntu Mono' 
plt.rcParams['font.size'] = 14 
plt.rcParams['axes.labelsize'] = 12 
plt.rcParams['axes.labelweight'] = 'bold' 
plt.rcParams['axes.titlesize'] = 12 
plt.rcParams['xtick.labelsize'] = 12 
plt.rcParams['ytick.labelsize'] = 12 
plt.rcParams['legend.fontsize'] = 12 
plt.rcParams['figure.titlesize'] = 12 
plt.rcParams['image.cmap'] = 'jet' 
plt.rcParams['image.interpolation'] = 'none' 
plt.rcParams['figure.figsize'] = (12, 10) 
plt.rcParams['axes.grid']=True
plt.rcParams['lines.linewidth'] = 2 
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue',
'xkcd:scarlet']

In [2]:
data = pd.read_csv('media.csv')

In [3]:
movie_data = data.loc[data.media_type == 'movie']
movie_data = movie_data[['media_type','id','title','overview','credits','genre_ids','top_cast']]
movie_data.head()

,media_type,id,title,overview,credits,genre_ids,top_cast
0,movie,27205,Inception,"Cobb, a skilled thief who commits corporate es...",525 525 556 2157773 2157775 41018 54211,28 878 12,6193 24045 3899 2524 27578
1,movie,155,The Dark Knight,Batman raises the stakes in his war on crime. ...,525 525 527 282 525 556 20643 10949 10951 5421...,18 28 80 53,3894 1810 3895 64 6383
2,movie,24428,The Avengers,When an unexpected enemy emerges and threatens...,12891 12891 10850 7624 7626 15277 41018 57027,878 28 12,3223 16828 103 74568 1245
3,movie,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",2710 2710 8529 20294 1385649,28 12 14 878,65731 8691 10205 32747 17647
4,movie,293660,Deadpool,The origin story of former Special Forces oper...,55252 7200 11092 10859 7624 7932 25292 71230 9...,28 12 35,10859 54882 1047649 51990 78452


In [4]:
tv_data = data.loc[data.media_type == 'tv']
tv_data = tv_data[['media_type','id','title','overview','credits','genre_ids','top_cast']]
tv_data.head()

,media_type,id,title,overview,credits,genre_ids,top_cast
491,tv,1399,Game of Thrones,Seven noble families fight for control of the ...,1406855 1878409 56746 1223799 33316 114404 122...,10765 18 10759,1223786 22970 239019 17286 1001657
492,tv,66732,Stranger Things,"When a young boy vanishes, a small town uncove...",1113659 3025043 937878 1718785 1179419 17825 1...,18 10765 9648,1920 35029 1356210 1442069 1653291
493,tv,1402,The Walking Dead,Sheriff's deputy Rick Grimes awakens from a co...,NaN,10759 18 10765,4886 31535 62220 84224 1252310
494,tv,63174,Lucifer,"Bored and unhappy as the Lord of Hell, Lucifer...",3211568 1215703 3211569 192944 1512022 3950 12...,80 10765,192944 37014 116474 21356 515875
495,tv,69050,Riverdale,"Set in the present, the series offers a bold, ...",NaN,80 18 9648,1680339 1136940 1721740 56730 1592855


In [5]:
M_overview = np.array(movie_data.overview)
M_credits = np.array(movie_data.credits)
M_genre_ids = np.array(movie_data.genre_ids)
M_top_cast = np.array(movie_data.top_cast)

In [6]:
T_overview = np.array(tv_data.overview)
T_credits = np.array(tv_data.credits)
T_genre_ids = np.array(tv_data.genre_ids)
T_top_cast = np.array(tv_data.top_cast)

In [7]:
M_overview_txt_data = M_overview.tolist()
M_overview_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
M_overview_embeddings = M_overview_model.encode(M_overview_txt_data, show_progress_bar=True)
M_overview = np.array(M_overview_embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
T_overview_txt_data = T_overview.tolist()
T_overview_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
T_overview_embeddings = T_overview_model.encode(T_overview_txt_data, show_progress_bar=True)
T_overview = np.array(T_overview_embeddings)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [9]:
M_credits = M_credits.astype(str)
M_credits_txt_data = M_credits.tolist()
M_credits_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
M_credits_embeddings = M_credits_model.encode(M_credits_txt_data, show_progress_bar=True)
M_credits = np.array(M_credits_embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [10]:
T_credits = T_credits.astype(str)
T_credits_txt_data = T_credits.tolist()
T_credits_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
T_credits_embeddings = T_credits_model.encode(T_credits_txt_data, show_progress_bar=True)
T_credits = np.array(T_credits_embeddings)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [11]:
M_genre_ids_txt_data = M_genre_ids.tolist()
M_genre_ids_model = SentenceTransformer('all-mpnet-base-v2')
M_genre_ids_embeddings = M_genre_ids_model.encode(M_genre_ids_txt_data, show_progress_bar=True)
M_genre_ids = np.array(M_genre_ids_embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [12]:
T_genre_ids_txt_data = T_genre_ids.tolist()
T_genre_ids_model = SentenceTransformer('all-mpnet-base-v2')
T_genre_ids_embeddings = T_genre_ids_model.encode(T_genre_ids_txt_data, show_progress_bar=True)
T_genre_ids = np.array(T_genre_ids_embeddings)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
M_top_cast_txt_data = M_top_cast.tolist()
M_top_cast_model = SentenceTransformer('all-mpnet-base-v2')
M_top_cast_embeddings = M_top_cast_model.encode(M_top_cast_txt_data, show_progress_bar=True)
M_top_cast = np.array(M_top_cast_embeddings)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
T_top_cast = T_top_cast.astype(str)
T_top_cast_txt_data = T_top_cast.tolist()
T_top_cast_model = SentenceTransformer('all-mpnet-base-v2')
T_top_cast_embeddings = T_top_cast_model.encode(T_top_cast_txt_data, show_progress_bar=True)
T_top_cast = np.array(T_top_cast_embeddings)

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

In [23]:
# input 5 titles
disney_movies = []
disney_movies.append(movie_data.index[movie_data['title'] == 'Brave'][0])
disney_movies.append(movie_data.index[movie_data['title'] == 'Finding Nemo'][0]) 
disney_movies.append(movie_data.index[movie_data['title'] == 'Up'][0])
disney_movies.append(movie_data.index[movie_data['title'] == 'The Incredibles'][0])
disney_movies.append(movie_data.index[movie_data['title'] == 'Zootopia'][0])
# print(movie_data.index[movie_data['title'] == 'Neighbors']) # 283
print(disney_movies)


[106, 45, 40, 60, 73]


In [24]:
overview_cos_sim = pd.DataFrame(cosine_similarity(M_overview)) # will hold the cos values
overview_list = []
for movie_loc in disney_movies:
    new_l = overview_cos_sim.loc[movie_loc].sort_values(ascending=False).index.tolist()[1:6]
    overview_list.append(new_l)
print(overview_list)

[[438, 255, 408, 455, 203], [67, 107, 194, 349, 75], [224, 50, 396, 335, 308], [236, 332, 358, 91, 465], [403, 347, 162, 150, 445]]


In [25]:
credits_cos_sim = pd.DataFrame(cosine_similarity(M_credits)) # will hold the cos values
credits_list = []
for movie_loc in disney_movies:
    new_l = credits_cos_sim.loc[movie_loc].sort_values(ascending=False).index.tolist()[1:6]
    credits_list.append(new_l)
print(credits_list)

[[34, 182, 266, 359, 296], [125, 40, 57, 71, 104], [57, 34, 182, 154, 104], [118, 71, 125, 91, 50], [269, 121, 53, 197, 77]]


In [26]:
genre_ids_cos_sim = pd.DataFrame(cosine_similarity(M_genre_ids)) # will hold the cos values
genre_ids_list = []
for movie_loc in disney_movies:
    new_l = genre_ids_cos_sim.loc[movie_loc].sort_values(ascending=False).index.tolist()[1:6]
    genre_ids_list.append(new_l)
print(genre_ids_list)

[[247, 323, 172, 276, 333], [375, 456, 364, 45, 154], [450, 114, 127, 174, 262], [118, 394, 146, 163, 458], [58, 409, 319, 104, 262]]


In [27]:
top_cast_cos_sim = pd.DataFrame(cosine_similarity(M_top_cast)) # will hold the cos values
top_cast_list = []
for movie_loc in disney_movies:
    new_l = top_cast_cos_sim.loc[movie_loc].sort_values(ascending=False).index.tolist()[1:6]
    top_cast_list.append( new_l)
print(top_cast_list)

[[395, 224, 247, 319, 257], [125, 1, 19, 213, 307], [80, 324, 474, 154, 257], [118, 179, 363, 185, 304], [445, 0, 122, 59, 111]]


In [28]:
cos_dict = {}

# print(overview_list)
# print(credits_list)
# print(genre_ids_list)
# print(top_cast_list)
# # disney_movies = [34, 40, 50, 205, 364]
for index, movie in enumerate(disney_movies):
    for loc in overview_list[index]:
        if loc in cos_dict:
            cos_dict[loc] += overview_cos_sim[movie][loc]
        else:
            cos_dict[loc] = overview_cos_sim[movie][loc]
    for loc in credits_list[index]:
        if loc in cos_dict:
            cos_dict[loc] += credits_cos_sim[movie][loc]
        else:
            cos_dict[loc] = credits_cos_sim[movie][loc]
    for loc in genre_ids_list[index]:
        if loc in cos_dict:
            cos_dict[loc] += genre_ids_cos_sim[movie][loc]
        else:
            cos_dict[loc] = genre_ids_cos_sim[movie][loc]
    for loc in top_cast_list[index]:
        if loc in cos_dict:
            cos_dict[loc] += top_cast_cos_sim[movie][loc]
        else:
            cos_dict[loc] = top_cast_cos_sim[movie][loc]
print(cos_dict)

{438: 0.8223501, 255: 0.81591934, 408: 0.8120443, 455: 0.7957825, 203: 0.7913141, 34: 1.8478808, 182: 1.8380721, 266: 0.90826464, 359: 0.9025749, 296: 0.8975799, 247: 1.7944002, 323: 0.93930197, 172: 0.9290969, 276: 0.9220711, 333: 0.91890705, 395: 0.8751012, 224: 1.6264843, 319: 1.81724, 257: 1.6913061, 67: 0.7760337, 107: 0.7758925, 194: 0.7680224, 349: 0.7644001, 75: 0.75370926, 125: 2.4812183, 40: 0.8398768, 57: 1.7603576, 71: 1.7286184, 104: 2.6727543, 375: 0.9999999, 456: 0.9999999, 364: 0.9999999, 45: 0.9999999, 154: 2.745438, 1: 0.73367417, 19: 0.69847965, 213: 0.69808334, 307: 0.678681, 50: 1.6317964, 396: 0.7440076, 335: 0.74397916, 308: 0.7383884, 450: 1.0000001, 114: 1.0000001, 127: 0.976465, 174: 0.9731595, 262: 1.9296043, 80: 0.8579495, 324: 0.8557178, 474: 0.8501815, 236: 0.8450403, 332: 0.8429265, 358: 0.8310951, 91: 1.7091525, 465: 0.8060254, 118: 2.8444219, 394: 0.9436372, 146: 0.94300973, 163: 0.90971136, 458: 0.9017452, 179: 0.89516747, 363: 0.8900774, 185: 0.888189

In [30]:
sorted_cos_dict = []
sorted_cos_dict = sorted(cos_dict.items(), key=lambda x:x[1], reverse=True)
# print(sorted_cos_dict[:10])
for m_id, cos_data in sorted_cos_dict[:10]:
    print(movie_data.title[m_id])

Incredibles 2
Monsters University
Cars
Finding Dory
Ice Age: Continental Drift
Inside Out
Toy Story 4
Rio
The Lego Movie
Monsters, Inc.
